In [ ]:
# SageMaker ML Deployment Notebook (Error-Free Version)
# -----------------------------------------------------
# This notebook trains a model, packages it, uploads it to S3, and deploys it to a SageMaker endpoint.
# It ensures compatibility by using scikit-learn 1.0.2 and correct model packaging.

# 1️⃣ Install compatible scikit-learn version
!pip install scikit-learn==1.0.2 --quiet

# 2️⃣ Import Libraries
import pandas as pd
import numpy as np
import joblib
import tarfile
import sagemaker
from sklearn.ensemble import RandomForestClassifier
from sagemaker.sklearn.model import SKLearnModel
from sagemaker import get_execution_role

# Initialize session and role
session = sagemaker.Session()
bucket = session.default_bucket()
role = get_execution_role()

# 3️⃣ Generate Sample Sensor Data
data = []
for _ in range(1000):
    temp = round(np.random.uniform(20, 40), 2)
    humidity = round(np.random.uniform(50, 90), 2)
    soil = round(np.random.uniform(5, 40), 2)
    irrigation = 1 if soil < 15 and temp > 30 else 0
    data.append([temp, humidity, soil, irrigation])

df = pd.DataFrame(data, columns=["temperature", "humidity", "soil_moisture", "irrigation_needed"])
df.to_csv("sensor_training_data.csv", index=False)

# 4️⃣ Train Model
X = df[["temperature", "humidity", "soil_moisture"]]
y = df["irrigation_needed"]

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X, y)

# Save model with protocol=4 for compatibility
joblib.dump(model, "irrigation_model.joblib", protocol=4)
print(" Model trained and saved")

# 5️⃣ Package Model for SageMaker
with tarfile.open("model.tar.gz", "w:gz") as tar:
    tar.add("irrigation_model.joblib", arcname="irrigation_model.joblib")
print(" Model packaged")

# 6️⃣ Upload Model to S3
s3_path = session.upload_data("model.tar.gz", bucket=bucket, key_prefix="models")
print(" Model uploaded to:", s3_path)

# 7️⃣ Deploy Model to Endpoint
model = SKLearnModel(
    model_data=s3_path,
    role=role,
    entry_point="predict.py",  # Ensure predict.py is in the same directory
    framework_version="1.0-1",
    py_version="py3"
)

predictor = model.deploy(
    instance_type="ml.m5.large",
    initial_instance_count=1
)
print(" Model deployed successfully")


